<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/ask_pubmed_tweet_overlap_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Training%20set/AskAPatient.fold-0.train.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Validation%20set/AskAPatient.fold-0.validation.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Testing%20set/AskAPatient_overlap_30.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [10]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

   0                1
0  0             Fear
1  0             Fear
2  0             Fear
3  0  scared to death
4  1   Abdominal rash


In [12]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [14]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [17]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'i',
 'be',
 'mg',
 'group',
 'study',
 'disease',
 'are',
 'treatment',
 'not',
 'pain',
 'after',
 'an',
 'it',
 'amp',
 'release',
 'have',
 'p',
 'gluten',
 'we',
 'free',
 'than',
 'effect',
 'all',
 'no',
 'compared',
 'new',
 'using',
 'more',
 'effects',
 'your',
 'groups',
 'two',
 'our',
 'its',
 'between',
 'both',
 'inflammatory',
 'time',
 'my',
 'about',
 'h',
 'these',
 'drugs',
 'significantly',
 'significant',
 'out',
 'use',
 'can',
 'anti',
 'but',
 'which',
 'day',
 'dose',
 'used',
 'has',
 'potassium',
 'results',
 'voltaren',
 'what',
 'placebo',
 'also',
 'there',
 'one',
 'glutenfree',
 'do',
 'may',
 'efficacy',
 's',
 'nt',
 'celiac',
 'ds',
 'treated',
 'm',
 'w',

In [18]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [19]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [20]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.186597,4.626398,0.250391,21:51


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.436688,4.290257,0.283650,21:41


In [22]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.103340,4.251094,0.287556,21:40


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.071471,4.222544,0.290662,21:28


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [25]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.683698,3.488134,0.437870,02:30


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.710276,2.976850,0.520710,02:46


In [27]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.287001,2.708366,0.566864,03:23


In [28]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.036551,2.521669,0.597633,03:08
1,1.846652,2.416235,0.620118,03:10


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.760375,2.307924,0.616568,03:27
1,1.602349,2.215948,0.628402,03:27


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.553587,2.138977,0.643787,03:16
1,1.420583,2.078921,0.648521,03:18


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.345745,2.080702,0.646154,03:03
1,1.326653,2.017823,0.660355,03:22


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.312039,2.010334,0.652071,03:13
1,1.209362,1.966309,0.650888,03:31


In [33]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.175089,1.947678,0.655621,03:28
1,1.210405,1.906295,0.657988,03:20
2,1.108154,1.881429,0.661538,03:23
3,1.027926,1.886383,0.663905,03:29


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.032417,1.910512,0.662722,03:06
1,1.089008,1.921585,0.656805,03:19
2,1.001599,1.887837,0.669823,03:07
3,0.945709,1.896953,0.666272,03:21


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.978484,1.873084,0.671006,03:08
1,0.984122,1.928080,0.669823,03:34
2,0.922647,1.928776,0.663905,03:10
3,0.890724,1.936124,0.671006,03:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.890204,1.944084,0.674556,03:23
1,0.955137,1.921835,0.675740,03:09
2,0.927154,1.936837,0.686391,03:11
3,0.855707,1.899578,0.689941,03:29


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.849388,1.967788,0.665089,03:15
1,0.891466,1.956928,0.681657,03:33
2,0.870691,1.959212,0.680473,03:15
3,0.857187,1.971548,0.681657,03:07


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.879065,1.987623,0.668639,03:16
1,0.906384,2.035431,0.675740,03:06
2,0.854945,1.930645,0.680473,03:06
3,0.838462,2.032336,0.685207,03:14


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.814473,1.991704,0.680473,03:14
1,0.841123,1.973006,0.682840,03:09
2,0.850415,1.996071,0.682840,03:33
3,0.770245,1.977106,0.681657,03:34


In [40]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.825919,2.007492,0.691124,03:26
1,0.790679,1.967543,0.691124,03:22


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.852738,2.046791,0.675740,03:24
1,0.810966,1.942632,0.680473,03:21


In [42]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.814165,2.000567,0.685207,03:26


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [44]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [45]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

872
872
[840, 265, 830, 561, 5, 830, 1026, 8, 8, 8, 8, 760, 10, 769, 137, 71, 817, 173, 27, 427, 805, 491, 129, 30, 30, 31, 32, 34, 265, 265, 880, 528, 289, 146, 1032, 1032, 49, 597, 49, 49, 49, 29, 232, 232, 232, 232, 245, 113, 375, 127, 71, 71, 289, 76, 76, 260, 182, 172, 817, 817, 190, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 260, 289, 289, 817, 37, 37, 80, 80, 80, 80, 359, 88, 89, 137, 391, 377, 91, 517, 135, 94, 94, 135, 135, 49, 805, 492, 98, 49, 100, 100, 100, 337, 100, 100, 490, 100, 168, 265, 805, 108, 88, 209, 470, 636, 111, 111, 470, 111, 111, 211, 111, 111, 427, 120, 1035, 817, 125, 125, 125, 125, 125, 817, 125, 127, 579, 129, 129, 289, 569, 492, 142, 140, 343, 415, 817, 111, 157, 52, 470, 165, 359, 817, 817, 168, 298, 172, 190, 864, 864, 864, 864, 79, 182, 37, 182, 182, 289, 427, 183, 183, 183, 183, 183, 817, 185, 187, 187, 817, 817, 817, 189, 190, 190, 190, 190, 190, 190, 190, 289, 289, 193, 197, 197, 214, 214, 214, 805, 245, 96, 245, 210, 210, 210, 210, 26

In [46]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

370
0.4243119266055046
